# **Lecture des Fichiers PDF**

In [ ]:
!pip install pypdf2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


^C


In [ ]:
!pip install textacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━ 4/5 [textacy]ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━ 4/5 [textacy]


In [1]:
import PyPDF2

def extract_text_from_pdf(pdf_file: str) -> [str]:
    with open(pdf_file, 'rb') as pdf:
        reader = PyPDF2.PdfReader(pdf, strict = False)
        pdf_text = []

        for page in reader.pages:
            content = page.extract_text()
            pdf_text.append(content)
        return pdf_text

ModuleNotFoundError: No module named 'PyPDF2'

In [ ]:
import re

def remove_https_links(text: str) -> str:
    # « https:// » suivi de caractères non-blancs jusqu’à un séparateur espace ou fin de ligne
    pattern = r"https://\S+"
    return re.sub(pattern, "", text)

In [ ]:
DORA_extract = extract_text_from_pdf("DORA doc.pdf")
NIST_extract = extract_text_from_pdf("NIST doc.pdf")

docs = []
for el in DORA_extract[3:]:
  docs.append(el)

docs.append(NIST_extract[2])
for el in NIST_extract[5:]:
  docs.append(el)

for el in docs:
  print(el)

Nettoyage des données

In [ ]:
#enlever les liens
docs_no_https = []
for text in docs:
  docs_no_https.append(remove_https_links(text))


#enlever les doubles allers à la ligne pour éviter que spacy forme mal les phrases
for el in docs_no_https:
  el.replace("\n\n", " ").replace("\n", " ")
for text in docs_no_https:
  print(text)


# SpaCy


Passage au propre: suppression des stop words

In [ ]:
#création de chunks adéquats pour le sentence transformer
import spacy

nlp = spacy.load("en_core_web_sm")

sentences = []

for page in docs_no_https:
  doc = nlp(page)
  page_sentences = list(doc.sents)
  for sentence in page_sentences:
    sentences.append(sentence)

for sentence in sentences:
  print(sentence)

print(len(sentences))

DORA: A COMPLETE GUIDE TO  
COMPLIANCE FOR THE FINANCIAL 
SECTOR
Looking for a practical guide to DORA compliance?
Here’s a 17-point checklist to 
get you ready for the Digital Operational Resilience Act, the EU’s core regulation 
for digital finance. 
 

This article is primarily intended for Chief Information Security Officers (CISOs), but may 
be useful for Data Protection Officers (DPOs) and legal departments.
It is the outcome of 
a personal work based on  the final text of DORA, which will be regularly quoted as 
source. 

Nevertheless, as exhaustive as they aim to be, these few pages cannot substitute for the 
necessary diligence of each and every one regarding compliance, nor for the expertise of 
any legal expert.
It is therefore up to each individual to ensure proper compliance, and this 
article shouldn’t be taken as anything more than what it is: a humble guide, written with 
caution and rigor.
Anyway, no more disclaimer, and let’s get to the heart of the matter.

The DORA 

In [ ]:
import spacy
import textacy
from spacy.lang.en.stop_words import STOP_WORDS

for sw in ("i.e.", "e.g."):
    STOP_WORDS.add(sw)
    nlp.vocab[sw].is_stop = True

chunked_sentences = []         # on part sur une liste vide

for sentence in sentences:     # sentences = liste de chaînes
    tokens = [token.text                     # ou tok si tu veux garder l’objet
              for token in sentence
              if not token.is_stop and not token.is_punct and not token.is_space and token.pos_ != "DATE"]         # filtre la ponctuation
    chunked_sentences.append(str(tokens))       # on stocke la liste de tokens


# aperçu rapide
for tokens in chunked_sentences:
    print(tokens)

['DORA', 'COMPLETE', 'GUIDE', 'COMPLIANCE', 'FINANCIAL', 'SECTOR', 'Looking', 'practical', 'guide', 'DORA', 'compliance']
['17', 'point', 'checklist', 'ready', 'Digital', 'Operational', 'Resilience', 'Act', 'EU', 'core', 'regulation', 'digital', 'finance']
['article', 'primarily', 'intended', 'Chief', 'Information', 'Security', 'Officers', 'CISOs', 'useful', 'Data', 'Protection', 'Officers', 'DPOs', 'legal', 'departments']
['outcome', 'personal', 'work', 'based', 'final', 'text', 'DORA', 'regularly', 'quoted', 'source']
['exhaustive', 'aim', 'pages', 'substitute', 'necessary', 'diligence', 'compliance', 'expertise', 'legal', 'expert']
['individual', 'ensure', 'proper', 'compliance', 'article', 'taken', 'humble', 'guide', 'written', 'caution', 'rigor']
['disclaimer', 'let', 'heart', 'matter']
['DORA', 'Regulation', '2022/2554', 'Digital', 'Operational', 'Resilience', 'Act', 'major', 'piece', 'European', 'Union', 'legislation', 'cybersecurity', 'financial', 'entities', 'banks', 'credit',

In [ ]:
for sentence in sentences:     # sentences = liste de chaînes
    tokens = [token.text                     # ou tok si tu veux garder l’objet
              for token in sentence
              if not token.is_stop and not token.is_punct and not token.is_space and token.pos_ != "DATE"]

# **Sentence Transformer**

In [ ]:
from sentence_transformers import SentenceTransformer

#model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

embeddings = model.encode(
    chunked_sentences,          # liste complète de strings
    batch_size=64,              # ajuste en fonction de ta VRAM
    convert_to_numpy=True,      # ou False si tu veux un tensor
    normalize_embeddings=True,  # important pour le cosinus
    show_progress_bar=True      # pratique dans un notebook
)
print(embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

[[ 0.01969885 -0.05735638 -0.04453557 ...  0.03668585  0.03204562
   0.05028627]
 [ 0.00822222 -0.092949   -0.07699606 ...  0.02382436  0.00105765
   0.04534156]
 [ 0.00148892 -0.00509916 -0.03651279 ...  0.06964087  0.05116727
   0.06795017]
 ...
 [-0.03734984  0.01799533 -0.10233047 ...  0.06699381  0.03611994
   0.02747624]
 [-0.02481949 -0.01884815 -0.08918921 ...  0.05463341  0.04564909
   0.00383523]
 [-0.02398247 -0.04053144 -0.1232065  ...  0.02214062  0.01929252
   0.04137316]]


In [ ]:
print(len(embeddings))

1003


In [ ]:
#transformation en dataframe & csv
import pandas as pd

df = pd.DataFrame(embeddings)
df.to_csv("embeddings.csv", index=False)

# **Evaluation**

In [ ]:
sentences_test = [
    "As you will see, some of the policies and procedures introduced by DORA are, to date, still a bit nebulous",
    "As previously seen, the goal of DORA is to elevate the digital operational resilienceof financial organizations.",
    "Another area that is foundational for vulnerability remediation prioritization are Known Exploited Vulnerability (KEV) lists.KEV lists are enumerations of vulnerabilities that have definitively been exploited in the past (e.g., [5]  and [6]).However, such lists may not be comprehensive, and prior to this research, metrology did not exist to measure their coverage.  "]

In [ ]:
emb_test = model.encode(
    sentences_test,
    batch_size=64,              # ajuste en fonction de ta VRAM
    convert_to_numpy=True,      # ou False si tu veux un tensor
    normalize_embeddings=True,  # important pour le cosinus
    show_progress_bar=True      # pratique dans un notebook
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def dists_cosine(sentences):
  for i in range()

In [ ]:
import numpy as np
from scipy.spatial.distance import cosine

N = 2
dists = []
dists_scalaire = []

for i in range(len(embeddings)):
    dist = cosine(emb_test[N], embeddings[i])
    dist_scalaire = abs(np.dot(emb_test[N], embeddings[i]))
    dists.append({'idx': i, 'dist': dist})
    dists_scalaire.append({'idx': i, 'dist': dist_scalaire})

# tri du plus proche au plus lointain
dists_sorted = sorted(dists, key=lambda x: x['dist'])
dists_scalaire_sorted = sorted(dists_scalaire, key = lambda x: x['dist'])
print(sentences_test[N])

# cinq meilleures correspondances
print("COSINE")
for item in dists_sorted[:15]:
    idx  = item['idx']
    dist = item['dist']
    print(f"{dist:.4f}  →  {sentences[idx]}")

print("PROD SCALAIRE")
for item in dists_scalaire_sorted[:15]:
  idx = item['idx']
  dist = item['dist']
  print(f"{dist:.4f} → {sentences[idx]}")

Another area that is foundational for vulnerability remediation prioritization are Known Exploited Vulnerability (KEV) lists.KEV lists are enumerations of vulnerabilities that have definitively been exploited in the past (e.g., [5]  and [6]).However, such lists may not be comprehensive, and prior to this research, metrology did not exist to measure their coverage.  
COSINE
0.3166  →  Another area that is foundational for vulnerability remediation prioritization are Known Exploited Vulnerability (KEV) lists.
0.3794  →  This 
covers then both the chance that the vulnerability was previously exploited (which in some cases is known to be 1.0 for vulnerabilities on a KEV list) and the chance that it will be exploited.  

0.3811  →  Known Exploited Vulnerability Lists   
KEV lists identify  vulnerabilities that are known to have been exploited in the past.
0.3940  →  If the vulnerability has a high LEV because many 
scanners are attempting to exploit it, but all in -scope systems have been h